## <span><h1 style = "font-family: garamond; font-size: 35px; font-style: normal; letter-spcaing: 4px; background-color: #f6f5f5; color :#fe346e; border-radius: 150px 150px; text-align:center">0. Library</h1></span>

In [ ]:
%%capture
!pip install -U ultralytics albumentations wandb ipywidgets iterative-stratification

In [ ]:
import os
import re
import random
import shutil
import pandas as pd
from collections import defaultdict
from sklearn.model_selection import train_test_split
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

import numpy as np
import torch
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import matplotlib.patches as patches

from albumentations import (
    HorizontalFlip, RandomRotate90, ShiftScaleRotate,
    RandomBrightnessContrast, HueSaturationValue, Compose
)

from ultralytics import YOLO


## <span><h1 style = "font-family: garamond; font-size: 35px; font-style: normal; letter-spcaing: 4px; background-color: #f6f5f5; color :#fe346e; border-radius: 150px 150px; text-align:center">1. Configuration</h1></span>

In [ ]:
def set_SEED(seed: int=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_SEED(42)

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">2. Training Yolo</h1></span>

In [ ]:
import os
import shutil
import pandas as pd
from ultralytics import YOLO
from PIL import Image
import torch
import gc
import warnings
import numpy as np

warnings.filterwarnings("ignore")

# Đường dẫn cấu hình
DATA_YAML = '/kaggle/input/vaipe-yaml/ai4vn.yaml'
PROJECT_DIR = '/kaggle/working/runs'
PRETRAINED_MODEL = 'yolov8x.pt'
INFERENCE_DEVICE = '0'  # Sử dụng 1 GPU

# Tham số huấn luyện (đã được điều chỉnh)
EPOCHS = 100            
BATCH_SIZE = 8    
IMG_SIZE = 1920      
AUGMENT = True 
MULTI_SCALE = False
LR0 = 0.001
LRF = 0.01
PRETRAINED = True
NAME = 'yolo11x_vehicle_detection'
COS_LR = True
SAVE = True
SAVE_PERIOD = 10
WORKERS = 4 
SEED = 42
DEVICE = '0,1'
OPTIMIZER = 'AdamW'
DROPOUT = 0.0
PATIENCE = 10
WEIGHT_DECAY = 0.0015
DROP_OUT = 0.1
PLOTS = True

CHECKPOINT_PATH = f'/kaggle/input/yolo11x-1024-range-100-epochs/last.pt'

# Khởi tạo mô hình
CONTINUE = False
if os.path.exists(CHECKPOINT_PATH) and CONTINUE:
    model = YOLO(CHECKPOINT_PATH)
    print(f"Đã tải checkpoint từ: {CHECKPOINT_PATH}")
else:
    model = YOLO(PRETRAINED_MODEL)
    print(f"Đã tải mô hình pretrained từ: {PRETRAINED_MODEL}")

# Đường dẫn tới tập validation và các file
VAL_IMAGE_FOLDER = '/kaggle/input/vaipe-dataset/content/ai4vn-vaipe-baseline/data/ai4vn/images/val'
PREDICT_OUTPUT_DIR = '/kaggle/input/vaipe-dataset/content/ai4vn-vaipe-baseline/data/ai4vn/images/test'
PREDICT_OUTPUT_PATH = os.path.join(PREDICT_OUTPUT_DIR, 'predict_val.txt')
os.makedirs(PREDICT_OUTPUT_DIR, exist_ok=True)

# Kiểm tra số lượng GPU
print(f"Số GPU đang sử dụng: {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

# Huấn luyện và đánh giá từng epoch
for epoch in range(1, EPOCHS + 1):
    print(f"==== Epoch {epoch}/{EPOCHS} ====")
    
    # Huấn luyện mô hình một epoch
    model.train(
        data=DATA_YAML,
        epochs=EPOCHS,                # Huấn luyện từng epoch
        batch=BATCH_SIZE,
        imgsz=IMG_SIZE,
        augment=AUGMENT,
        multi_scale=MULTI_SCALE,
        lr0=LR0,
        lrf=LRF,
        pretrained=PRETRAINED,
        name=NAME,
        project=PROJECT_DIR,
        cos_lr=COS_LR,
        save=SAVE,
        save_period=SAVE_PERIOD,
        workers=WORKERS,
        seed=SEED,
        device=DEVICE,
        optimizer=OPTIMIZER,
        dropout=DROPOUT,
        plots=PLOTS,
        weight_decay=WEIGHT_DECAY,
        patience=PATIENCE,
        # resume=True
    )
    
    print(f"==== Kết thúc Epoch {epoch} ====\n")
    
    # Giải phóng bộ nhớ GPU để tránh tăng dần
    torch.cuda.empty_cache()
    gc.collect()

# Giải phóng bộ nhớ cuối cùng
torch.cuda.empty_cache()
gc.collect()


# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">3. Prediction</h1></span>

In [ ]:
import os
from ultralytics import YOLO
import torch
import zipfile
from tqdm import tqdm

# Đặt cấu hình CUDA để tránh phân mảnh bộ nhớ
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()

# Đường dẫn tới thư mục chứa hình ảnh test
TEST_IMAGES_DIR = '/kaggle/input/vaipe-dataset/content/ai4vn-vaipe-baseline/data/ai4vn/images/test'

# Đường dẫn tới mô hình đã huấn luyện
MODEL_PATH = '/kaggle/input/yolo11x-1024-best-vaipe/best (1).pt'

# Tải mô hình
model = YOLO(MODEL_PATH)

# Lấy danh sách hình ảnh
image_files = [os.path.join(TEST_IMAGES_DIR, img) for img in os.listdir(TEST_IMAGES_DIR) if img.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Đường dẫn tới file predict.txt
PREDICT_TXT_PATH = '/kaggle/working/predict.txt'

# Ghi kết quả vào predict.txt
with open(PREDICT_TXT_PATH, 'w') as f:
    for image_file in tqdm(image_files, desc='Predicting...'):
        # Dọn dẹp bộ nhớ trước mỗi lần xử lý
        torch.cuda.empty_cache()
        
        # Chạy dự đoán từng ảnh với kích thước
        result = model.predict(
            source=image_file,
            imgsz=1024,
            conf=0.25,
            iou=0.45,
            save=False,
            verbose=False,
            device='0',
            stream=False,
            batch=1
        )[0]

        # Lấy kích thước ảnh gốc
        img_height, img_width = result.orig_shape

        # Ghi thông tin bounding box vào file
        img_path = result.path
        image_id = os.path.basename(img_path)
        boxes = result.boxes
        if boxes is None or len(boxes) == 0:
            continue
        
        xywh = boxes.xywh.cpu().numpy()  # x_center, y_center, width, height
        scores = boxes.conf.cpu().numpy()  # Confidence scores
        class_ids = boxes.cls.cpu().numpy()  # Class IDs

        for box, score, class_id in zip(xywh, scores, class_ids):
            x_center, y_center, width, height = box
            # Tính xmin và ymin đã scale
            x_center = (x_center) / img_width
            y_center = (y_center) / img_height
            width /= img_width
            height /= img_height
            # Ghi vào file
            f.write(f"{image_id} {int(class_id)} {x_center} {y_center} {width} {height} {score}\n")

# Đường dẫn tới file zip đầu ra
SUBMISSION_ZIP_PATH = '/kaggle/working/predict.zip'

# Nén file predict.txt thành submission.zip
with zipfile.ZipFile(SUBMISSION_ZIP_PATH, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(PREDICT_TXT_PATH, arcname='predict.txt')

print(f"Đã lưu kết quả dự đoán tại: {PREDICT_TXT_PATH}")
print(f"Đã nén file predict.txt thành: {SUBMISSION_ZIP_PATH}")


# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">4. Evaluation</h1></span>

In [1]:
from ultralytics import YOLO

# Đường dẫn mô hình và file cấu hình
MODEL_PATH = '/kaggle/input/yolo11x-1024-best-vaipe/best.pt'
DATA_YAML = '/kaggle/input/data-yaml/ai4vn.yaml'

# Tải mô hình
model = YOLO(MODEL_PATH)

# Đánh giá trên tập test
results = model.val(
    data=DATA_YAML,   # File cấu hình .yaml (chỉ định tập test)
    split='test',
    conf=0.25,        # Ngưỡng confidence
    iou=0.45,          # Ngưỡng IoU
)

# In các metrics chính (giá trị trung bình nếu là mảng)
precision = results.box.mp.mean() if isinstance(results.box.mp, np.ndarray) else results.box.mp
recall = results.box.mr.mean() if isinstance(results.box.mr, np.ndarray) else results.box.mr
f1_score = results.box.f1.mean() if isinstance(results.box.f1, np.ndarray) else results.box.f1
map50 = results.box.map50.mean() if isinstance(results.box.map50, np.ndarray) else results.box.map50
map_50_95 = results.box.map.mean() if isinstance(results.box.map, np.ndarray) else results.box.map

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")
print(f"mAP@50: {map50:.4f}")
print(f"mAP@[50:95]: {map_50_95:.4f}")


ModuleNotFoundError: No module named 'ultralytics'